In [ ]:
import numpy as  np
import pandas as pd
import trackpy as tp
import matplotlib.pyplot as plt
import os
import numba
import seaborn as sns
from IPython.display import display, HTML
import helperfunctions # this is my own python file
from stdev_emsd import stdev_emsd, multi_stdev_emsd # this is my own python file
import scipy

sns.set_context('notebook')

In [ ]:
# Please give all the files you want to combine. I assume these have not been drift-subtracted.
filename_list = [r"C:\Users\vhorowit\Desktop\Sara Conti\Analysis\Data Taken 2024-07-22, 2024-07-22_ND4_1.avicontrol_tracer.pkl",
                 r"C:\Users\vhorowit\Desktop\Sara Conti\Analysis\Data Taken 2024-07-22, 2024-07-22_ND4_2.avicontrol_tracer.pkl"]

# please give the fps for all these files.
fps =     # for example, 50
# We cannot combine these files unless the fps are all the same.

# please give the microns per pixel for all these files.
scaling_list = # for example, [1,1]

# Do you want to calculate the emsd of each file? or just calculate the combined emsd of all the files
calc_each_emsd = True
# do you want to see all the drift plots?
verbose_plots = True

In [ ]:
# stderr = stdev / sqrt(N)
def calc_stderr_for_emsd_df(emsd_df):
        # Calculate standard error from standard deviation
        stderr_msd = emsd_df.std_msd / np.sqrt(emsd_df.N)
        return emsd_df.assign(stderr_msd=stderr_msd)

In [ ]:
# Create emsd dataframes with the following columns:
    # lagt = lag time, 
    # msd = ensemble mean square displacement, 
    # N = effective number of independent measurements, 
    # std_msd = standard deviation of mean square displacement
    # stderr_msd = standard error of mean square displacement
    # weight_msd = the weight to use in a weighted average of emsds

trajectories_list = [] # initialize list of dataframes
emsd_list = []
for filepath, scaling in zip(filename_list, scaling_list):
    traj_drifty = pd.read_pickle(filepath) #in pixels and frames
    drift = tp.compute_drift(traj_drifty,smoothing=5) #in pixels and frames
    traj = tp.subtract_drift(traj_drifty, drift)
    traj=traj.reset_index(drop=True)
    if verbose_plots:
        drift.plot();
        plt.title('Drift\n' + os.path.basename(filepath));
        plt.figure();
        tp.subpx_bias(traj_drifty);
    trajectories_list.append(traj) # build list of trajectory files
    
    if calc_each_emsd:
        emsd_full_df = stdev_emsd(traj, mpp=scaling, fps = fps, detail = True)#inputs pixels and frames, outputs microns and seconds
        emsd_df = emsd_full_df[['lagt', 'msd', 'N', 'std_msd']]
        emsd_df = calc_stderr_for_emsd_df(emsd_df)
        
        emsd_list.append(emsd_df)   

if calc_each_emsd:
    # Create big dataframe named big_df to organize all these trajectories and their emsds.
    big_df = pd.DataFrame({'paths':filename_list, 
                           'files': [os.path.basename(filepath) for filepath in filename_list],   
                           'scaling': scaling_list, 
                           'trajectory': trajectories_list, 
                           'emsd': emsd_list})
    display(big_df)

In [ ]:
multi_emsd = multi_stdev_emsd(trajectories_list,scaling_list, fps, detail=True)#inputs pixels and frames, outputs microns and seconds
multi_emsd = calc_stderr_for_emsd_df(multi_emsd)

multi_emsd

In [ ]:
if calc_each_emsd:
    for emsd, file in zip(big_df.emsd, big_df.files):
        plt.plot(emsd.lagt, emsd.msd, linewidth=.5, label=file)

plt.loglog(multi_emsd.lagt, multi_emsd.msd, label="combined emsd")
plt.errorbar(multi_emsd.lagt, 
             multi_emsd.msd, 
             yerr = multi_emsd.std_msd, 
             capthick=0, 
             alpha = 0.7,
             linewidth=.3,
             label="combined emsd with standard deviation")

plt.errorbar(multi_emsd.lagt, 
             multi_emsd.msd, 
             yerr = multi_emsd.stderr_msd, 
             capthick=0, 
             alpha = 0.7,
             linewidth=1,
             label="combined emsd with standard error")

plt.xlabel('lag time')
plt.ylabel('ensemble MSD')

plt.legend(bbox_to_anchor=(1, 1), loc='upper left');

In [ ]:
# You could pickle multi_emsd.

In [ ]:
### Filter out small displacements from the curvefit.
threshold = 0.1 ## Might need to adjust this!


multi_emsd_filtered = multi_emsd[multi_emsd.msd >= threshold]

In [ ]:
print('Calculating best fit, using linear fit in log space')

res = tp.utils.fit_powerlaw(multi_emsd_filtered.set_index('lagt').msd, plot = False)  # performs linear best fit in log space, plots
display(res.transpose().msd)

plt.loglog(multi_emsd.lagt,multi_emsd.msd, 'k.')
plt.errorbar(multi_emsd.lagt, 
             multi_emsd.msd, 
             yerr = multi_emsd.stderr_msd, 
             capthick=0, 
             alpha = 0.7,
             linewidth=2,
             label="combined emsd with standard error")

plt.xlabel('lag time')
plt.ylabel('ensemble MSD')

def powerlaw(t, A, n):
    return (A * (t**n))

plt.loglog(multi_emsd.lagt, powerlaw(multi_emsd.lagt, res.A.msd, res.n.msd ), linewidth = 2, label='linear fit')

plt.legend(bbox_to_anchor=(1, 1), loc='upper left');

In [ ]:
print('Calculating best fit, using power law fit in linear space')

params, pcov = scipy.optimize.curve_fit(powerlaw, multi_emsd_filtered.lagt, multi_emsd_filtered.msd, 
                         p0=[res.A.msd, res.n.msd], # use previous results as initial guess
                         sigma=multi_emsd_filtered.std_msd,
                         absolute_sigma=True) 
# absolute_sigma =  True, so sigma is used in an absolute sense and the estimated parameter covariance pcov reflects these absolute values.

yfit = powerlaw(multi_emsd.lagt,params[0],params[1])

plt.loglog(multi_emsd.lagt,multi_emsd.msd, 'k.')
plt.errorbar(multi_emsd.lagt, 
             multi_emsd.msd, 
             color='k',
             yerr = multi_emsd.stderr_msd, 
             ecolor = 'k',
             capthick=0, 
             alpha = 0.5,
             linewidth=2,
             label="combined emsd with standard error")

plt.loglog(multi_emsd.lagt, yfit, label="power law fit",  color='m', linewidth = 2)

plt.xlabel('lag time $\Delta{}t$ [s]')
plt.ylabel(r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]')
#pylab.axes().set_aspect('equal', 'datalim')
#plt.title(moviename + '\ncurve fit')

#print 'Fit to power law, MSD ~ A * $\Delta$t$^n$.'
#print 'Coefficient A = ' + str(params[0]) + ' ± ' + str(pcov[0,0])
#print 'Exponent n = ' + str(params[1]) + ' ± ' + str(pcov[1,1])

plt.text(0.02,30, 
        'Fit to power law, MSD = A * $\Delta$t$^n$.'
         + '\nExponent n = {:.5} $\pm$ {:.3f}.'.format(params[1],np.sqrt(pcov[1,1]))
         + '\nCoefficient A = {:.5} $\pm$ {:.3f}.'.format(params[0],np.sqrt(pcov[0,0])))


# The entries on the diagonal of the covariance matrix \Sigma are 
# the variances of each element of the vector \mathbf{X}.
#  source: https://en.wikipedia.org/wiki/Covariance_matrix#Generalization_of_the_variance

plt.legend(bbox_to_anchor=(1, 1), loc='upper left');

In [ ]:
plt.imshow(pcov, cmap = 'gray')
plt.title('covariance matrix')
print('If the lower left or upper right squares are light-colored then we have a curvefitting problem. Tell Viva if that happens.')